In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('../data/anime.csv')

In [3]:
df.drop(labels=['Name',"English name","Japanese name",'Aired','Producers','Licensors','Duration','Members','Watching','On-Hold'], axis='columns', inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MAL_ID         17562 non-null  int64 
 1   Score          17562 non-null  object
 2   Genres         17562 non-null  object
 3   Type           17562 non-null  object
 4   Episodes       17562 non-null  object
 5   Premiered      17562 non-null  object
 6   Studios        17562 non-null  object
 7   Source         17562 non-null  object
 8   Rating         17562 non-null  object
 9   Ranked         17562 non-null  object
 10  Popularity     17562 non-null  int64 
 11  Favorites      17562 non-null  int64 
 12  Completed      17562 non-null  int64 
 13  Dropped        17562 non-null  int64 
 14  Plan to Watch  17562 non-null  int64 
 15  Score-10       17562 non-null  object
 16  Score-9        17562 non-null  object
 17  Score-8        17562 non-null  object
 18  Score-7        17562 non-n

this is code for computing baseline for weights calculation 

In [8]:
df['Score-1']

0         1580.0
1          379.0
2          533.0
3          131.0
4           27.0
          ...   
17557    Unknown
17558    Unknown
17559    Unknown
17560    Unknown
17561    Unknown
Name: Score-1, Length: 17562, dtype: object

In [5]:
df.describe()

,MAL_ID,Popularity,Favorites,Completed,Dropped,Plan to Watch
count,17562.000000,17562.000000,17562.000000,1.756200e+04,17562.000000,17562.000000
mean,21477.192347,8763.452340,457.746270,2.209557e+04,1176.599533,8199.831227
std,14900.093170,5059.327278,4063.473313,9.100919e+04,4740.348653,23777.691963
min,1.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000
25%,5953.500000,4383.500000,0.000000,1.110000e+02,37.000000,112.000000
50%,22820.000000,8762.500000,3.000000,8.175000e+02,77.000000,752.500000
75%,35624.750000,13145.000000,31.000000,6.478000e+03,271.000000,4135.500000
max,48492.000000,17565.000000,183914.000000,2.182587e+06,174710.000000,425531.000000


In [11]:
score_columns = ['Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
                 'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1']

In [12]:
df = df.dropna(subset=score_columns)

In [19]:
df[score_columns] = df[score_columns].replace('Unknown', np.nan)

In [27]:
df = df.dropna(subset=score_columns)

In [28]:
df[score_columns] = df[score_columns].astype(float).astype(int)

In [29]:
df['num_scored'] =  df[score_columns].sum(axis=1)

In [33]:
print(df['num_scored'].describe())
print("\nPercentiles:")
print(df['num_scored'].quantile([0.25, 0.3,  0.5, 0.55, 0.6,0.75, 0.8,  0.9]))

count    1.343600e+04
mean     2.320095e+04
std      8.466407e+04
min      2.000000e+01
25%      3.387500e+02
50%      1.595000e+03
75%      9.718750e+03
max      1.826691e+06
Name: num_scored, dtype: float64

Percentiles:
0.25      338.75
0.30      463.00
0.50     1595.00
0.55     2125.50
0.60     2921.00
0.75     9718.75
0.80    15494.00
0.90    48631.50
Name: num_scored, dtype: float64


this results (percentiles) will be used as baseline

In [191]:
numeric_cols = [
    'Score', 'Episodes', 'Ranked',
    'Score-10','Score-9','Score-8','Score-7','Score-6',
    'Score-5','Score-4','Score-3','Score-2','Score-1'
]

In [192]:
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [193]:
print(df[numeric_cols].isna().sum())

In [194]:
df["Episodes"] =df['Episodes'].fillna(0)

In [195]:
score_cols = ['Score-10','Score-9','Score-8','Score-7','Score-6',
              'Score-5','Score-4','Score-3','Score-2','Score-1']
df[score_cols] = df[score_cols].fillna(0)

In [ ]:
print(df[numeric_cols].isna().sum())

In [197]:
cat = df.select_dtypes(include='O').keys()

cat

In [198]:
def preprocess(df):
    df = df.copy()
    df = df.dropna(subset=['Score'])
    
    categorical_cols = ['Studios', 'Genres', 'Rating', 'Source', 'Type']
    for col in categorical_cols:
        df[col] = df[col].fillna('Unknown')
        df[col] = df[col].replace('', 'Unknown')
    
    numeric_cols = ['Episodes', 'Ranked']
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())
    return df

In [199]:
df = preprocess(df)

In [200]:
# Get dummy variables for each unique genre
genre_dummies = df['Genres'].str.get_dummies(sep=', ')

# Merge with original dataframe
df = pd.concat([df, genre_dummies], axis=1)

In [ ]:
df.info()

In [202]:
df[['Season','Year']] = df['Premiered'].str.split(' ', expand=True)
df.drop(columns=['Season'], inplace=True)
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

In [203]:
if 'Year' in df.columns:
    df['Year'] = 2025 - df['Year']

In [204]:
if 'Year' in df.columns:
        df['age_category'] = pd.cut(
            df['Year'],
            bins=[-1, 2, 5, 10, 20, np.inf],
            labels=['new', 'recent', 'modern', 'old', 'classic']
        )
        df = pd.get_dummies(df, columns=['age_category'], drop_first=True)
        df = df.drop(columns=['Year'])

In [205]:
if 'Episodes' in df.columns:
        df['episode_cat'] = pd.cut(
            df['Episodes'],
            bins=[0,1,10,18,26,57,np.inf],
            labels=['single', 'short', 'one_season','two_season', 'long', 'very_long'] # 1, 2-10, 11-18, 18-26,25-57,58-
        )
        df = pd.get_dummies(df, columns=['episode_cat'], drop_first=True)
        df = df.drop(columns=['Episodes'])

In [ ]:
df.info()

In [207]:
freq = df["Type"].value_counts()
print("Printing the frequency")
display(freq)

In [208]:
freq = df['Rating'].value_counts()
display(freq)

In [209]:
from sklearn.preprocessing import OneHotEncoder

In [210]:
categorical_columns = ['Type', 'Rating']

In [211]:
encoder = OneHotEncoder(categories='auto')

In [212]:
onehot_encoded = encoder.fit_transform(df[categorical_columns]).toarray()

In [213]:
one_hot_df = pd.DataFrame(onehot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

In [ ]:
display(one_hot_df)

In [215]:
type_dummies = pd.get_dummies(df['Type'], prefix='Type')
df = pd.concat([df, type_dummies], axis=1)

In [216]:
df[['Rating_index','Rating_info']] = df['Rating'].str.split(' - ', expand=True)
df.drop(columns=['Rating_info'], inplace=True)

rating_dummies = pd.get_dummies(df['Rating_index'], prefix='Rating')
df = pd.concat([df, rating_dummies], axis=1)

In [217]:
df.head()

In [218]:
df.drop(labels=['Genres', 'Type', 'Type_TV', 'Rating', 'Premiered', 'Rating_index', 'Rating_PG-13','Rating_Unknown'], axis='columns', inplace=True)

In [221]:
# df.drop(labels=['Rating_Unknown'], axis='columns', inplace=True)

In [222]:
top_studios = df['Studios'].value_counts().nlargest(10).index
df['Studios'] = df['Studios'].apply(lambda x: x if x in top_studios else 'Other')

In [ ]:
display(df['Studios'].value_counts().head(10))

In [224]:
studio_dummies = pd.get_dummies(df['Studios'], prefix='Studio')
df = pd.concat([df, studio_dummies], axis=1)

In [ ]:
df.info()

In [226]:
df.drop(labels=['Studios', 'Studio_Unknown', 'Unknown'], axis='columns', inplace=True)

In [227]:
df.drop(labels=[ 'Source'], axis='columns', inplace=True)

In [228]:
df.info()

In [229]:
bool_cols = df.select_dtypes('bool').columns

df[bool_cols] = df[bool_cols].astype(int)

In [230]:
X = df.drop(columns=['Score', 'MAL_ID'])
score_related_cols = ['Score-10','Score-9','Score-8','Score-7',
                      'Score-6','Score-5','Score-4','Score-3','Score-2','Score-1', 'Ranked']

X = X.drop(columns=score_related_cols)
y = df['Score']

In [231]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.info()

In [233]:
scaler = MinMaxScaler()

numeric_cols = ['Popularity', 'Favorites', 'Completed', 'Dropped', 'Plan to Watch']
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])

In [234]:
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [235]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

model = RandomForestRegressor(
    n_estimators=200, random_state=42, n_jobs=-1
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))


In [ ]:
X.info()

In [236]:
import matplotlib.pyplot as plt
import pandas as pd

importances = pd.Series(model.feature_importances_, index=X.columns)
importances.nlargest(20).plot(kind='barh', figsize=(8,6))
plt.title("Top-20 Feature Importances")
plt.show()

In [237]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(model, X, y, cv=5, scoring='r2', n_jobs=-1)
print("R2:", scores)
print("Mean R2:", np.mean(scores))